In [17]:
from bs4 import BeautifulSoup, SoupStrainer
import httplib2
import re
import pandas as pd
from itertools import chain
import time 

In [25]:
# setting up dataframe with columns required
scp_main_df = pd.DataFrame(columns=['code', 'title', 'text', 'image captions', 'link', 'rating', 'state'], index=None)

In [23]:
http = httplib2.Http()

In [29]:
# the page links all follow pattern and can loop through nums to string just zero padded

# note from 2235 onwards there are a couple that do not have the standard codes in directory pages, will keep the standard code for now
# also some marked as missing from site or along those lines, probably added placeholder text, may want to add flag for easy filtering out

for n in range(1,7000):
  # timeout to limit load on the site (based on api requests per minute limit)
  if n % 240 == 0:
    print(n)
    time.sleep(60)

  # getting the number as a string in the way need to get url
  num_as_str = '{}{}'.format('0' * (3 - len(str(n))), str(n)) if n < 1000 else str(n)

  # requesting page for current/target scp, parsing with beautifulsoup
  url = 'https://scp-wiki.wikidot.com/scp-{}'.format(num_as_str)
  status, response = http.request(url)
  soup = BeautifulSoup(response, 'html.parser')

  captions = []
  state = None

  if soup.find('div', {'id': 'u-adult-warning'}) != None:
    url = 'https://scp-wiki.wikidot.com/adult:scp-{}/noredirect/true'.format(num_as_str)

    status, response = http.request(url)
    soup_adult = BeautifulSoup(response)

    state = "age restricted"

    rating_tag = soup_adult.find(class_ = 'number prw54353')

    if rating_tag != None:
      rating = rating_tag.text  

    text = [i for i in soup_adult.find(id = 'page-content').find_all('p')]
  else:
    # getting text for the scp within the main content div
    text = [i for i in soup.find(id = 'page-content').find_all('p')]
    

  # p tags include image captions which are excluded as below, may be other similar cases left in
  for i in text:
    if hasattr(i.parent, 'class'):
      if i.parent.get("class") == 'scp-image-caption':
        text.remove(i)
        captions.append(i)

  # getting text from result set
  text = [i.text for i in text]

  # getting the rating - a few do not have this primarily the ones that are 'blocked' or similar
  if rating == None: # accounting for age restricted ones that already have ratings for w different method
    rating = soup.find(class_ = 'number prw54353')
    if rating == None:
      pass
    else:
      rating = rating.text

  # accounting for different directory pages (used to get title) based on number - getting correct url below
  if n < 1000:
    status, response = http.request('https://scp-wiki.wikidot.com/scp-series')
  else:
    status, response = http.request('https://scp-wiki.wikidot.com/scp-series-{}'.format(str(int(n//1000)+1)))

  # requesting directory page
  soup = BeautifulSoup(response, 'html.parser')

  # some logic to get the text title (not scp number code) 
  # using next or previous siblings and numbers where needed as some have no code on the directory page or multiple codes
  if soup.find('a', text='SCP-{}'.format(num_as_str)) == None:
    # checks here due to cases where no previous or next sibling for one of the elements involved
    if soup.find('a', text='SCP-{}'.format(str(n-1))) != None and soup.find('a', text='SCP-{}'.format(str(n-1))).parent.find_next_sibling() != None:
      title = soup.find('a', text='SCP-{}'.format(str(n-1))).parent.find_next_sibling().text.split(" - ")[-1]
    else:
      title = soup.find('a', text='SCP-{}'.format(str(n+1))).parent.find_previous_sibling().text
      # removing the code part of titles that do have a code here
      if ' - ' in title:
        title = title.split(" - ")[-1]
    # could potentially add both codes for those that have two
  # basic title finder for most cases
  else:
    title = soup.find('a', text='SCP-{}'.format(num_as_str)).parent.text.split(" - ")[-1]
  
  # quotes to preserve commas and not cause csv issues - joining up p tags with newlines
  text = "\"{}\"".format(" \n ".join(i for i in text))

  # setting status - active, deleted, blocked, age restricted, clearance required
  if state:
    pass
  elif "This page may have been moved or deleted" in text:
    state = "deleted"
  elif "[Blocked]" in title:
    state = "blocked"
  

  info = ['SCP-{}'.format(num_as_str), "\"{}\"".format(title), text, captions, url, rating, state]
  scp_main_df.loc[len(scp_main_df)] = info



KeyboardInterrupt: ignored

In [30]:
scp_main_df.head

<bound method NDFrame.head of         code                                          title  ... rating    state
0    SCP-001         "Awaiting De-classification [Blocked]"  ...   +682  blocked
1    SCP-002                            "The "Living" Room"  ...   +682     None
2    SCP-003                       "Biological Motherboard"  ...   +682     None
3    SCP-004               "The 12 Rusty Keys and the Door"  ...   +682     None
4    SCP-005                                 "Skeleton Key"  ...   +682     None
..       ...                                            ...  ...    ...      ...
99   SCP-100            ""Jamaican Joe's Junkyard Jubilee""  ...   +682     None
100  SCP-101                                   "Hungry Bag"  ...   +682     None
101  SCP-102  "Property of Marshall, Carter, and Dark Ltd."  ...   +682     None
102  SCP-103                         "The Never-Hungry Man"  ...   +682     None
103  SCP-104                              "The Lonely Ball"  ...   +682     Non

In [ ]:
scp_main_df.to_csv('scp6999.csv', index=False) 